# Data Generation for Topic Labeling using Top2Vec

by Andreas Sünder

## Setup

In [ ]:
dataset_name    = 'wikipedia'
subset_name     = '20220301.simple'
split           = 'train'

emb_model_id = 'jinaai/jina-embeddings-v2-small-en'

## Load Data

In [ ]:
from datasets import load_dataset
dataset = load_dataset(dataset_name, subset_name, split=split)

## Setup Model

In [ ]:
from transformers import AutoModel
model = AutoModel.from_pretrained(emb_model_id, trust_remote_code=True)

## Convert documents to vectors

In [ ]:
def process_row(row):
  row['embedding'] = model.encode(row['text'])
  return row

dataset_processed = dataset.map(process_row)

In [ ]:
dataset_processed = dataset_processed.remove_columns(['url', 'text'])

## UMAP

In [ ]:
from umap import UMAP

umap_embeddings = UMAP(
  n_neighbors=15, n_components=5, metric='cosine', verbose=True
).fit_transform(embeddings)

## HDBSCAN

In [ ]:
from hdbscan import HDBSCAN

cluster = HDBSCAN(
  min_cluster_size=15,
  metric='euclidean',
  cluster_selection_method='eom'
).fit(umap_embeddings)

## Visualization

In [ ]:
import matplotlib.pyplot as plt

umap_data = UMAP(
  n_neighbors=15, n_components=2, metric='cosine', verbose=True
).fit_transform(embeddings)

In [ ]:
result = pd.DataFrame(umap_data, columns=['x', 'y'])
result['labels'] = cluster.labels_
outliers = result.loc[result.labels == -1, :]
clustered = result.loc[result.labels != -1, :]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(outliers.x, outliers.y, color='#BDBDBD', s=0.5)
ax.scatter(clustered.x, clustered.y, c=clustered.labels, s=1, cmap='hsv_r')
plt.show()

## Topic Extraction

In [ ]:
docs_df = pd.DataFrame(df_filtered, columns=['title', 'content'])
docs_df['topic'] = cluster.labels_
docs_df = docs_df[docs_df.topic != -1]
docs_per_topic = docs_df.groupby(['topic'], as_index=False).agg({'content': ' '.join})

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count
  
tf_idf, count = c_tf_idf(docs_per_topic.content.values, m=len(df))

In [ ]:
def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names_out()
    labels = list(docs_per_topic.topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)

In [ ]:
for topic in range(len(top_n_words)):
  print(', '.join([word[0] for word in top_n_words[topic][:10]]))